<a href="https://colab.research.google.com/github/thaisbeham/Applied_deep_learning/blob/main/Assigment%202/Assigment2_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM - Fake News detection

Fake - 0
Real - 1

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
import tensorflow as tf
import pandas as pd

# from utils import merge_datasets, smaller_set
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from keras.layers import (
    Embedding,
    Dense,
    LSTM,
    Dropout,
    Flatten,
    BatchNormalization,
    Conv1D,
    GlobalMaxPooling1D,
    MaxPooling1D,
    GlobalAveragePooling1D,
)
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

# from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import string
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras import callbacks
from keras.utils import plot_model

nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Functions

In [ ]:



def read_news(path, column_text_name):
    news = pd.read_csv(path)
    news[column_text_name] = news[column_text_name].astype(str)
    if column_text_name != "text":
        news.rename(columns={column_text_name: "text"}, inplace=True)

    return news


def merge_split_datasets(real, fake, n_testsize=0.15):

    # add column label
    real["label"] = 1
    fake["label"] = 0

    real = real[["text", "label"]]
    fake = fake[["text", "label"]]

    # merge real and fake
    # use only the sentence column on fake
    merged = pd.DataFrame(real.append(fake, ignore_index=True))
    merged = merged.dropna()

    # shuffle dataset
    merged = merged.sample(frac=1, random_state=1, ignore_index=True)  # .reset_index()

    merged["text"].dropna(inplace=True)
    merged["text"] = merged["text"].astype(str)

    # remove stopwords
    tokens = merged["text"].apply(word_tokenize)
    tokens = tokens.astype(str)
    tokens = tokens.apply(str)
    stop_words = set(stopwords.words("english"))
    filtered_sentence = [w for w in tokens if not w.lower() in stop_words]

    X_train, X_test, Y_train, Y_test = train_test_split(
        filtered_sentence, merged["label"], test_size=n_testsize, random_state=1
    )

    X_train, X_valid, Y_train, Y_valid = train_test_split(
        X_train, Y_train, test_size=0.15, random_state=42
    )

    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test


def RNN(max_tokens, max_len, dropout):
    inputs = Input(name="inputs", shape=[max_len])
    layer = Embedding(max_tokens, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name="FC1")(layer)
    layer = Activation("relu")(layer)
    layer = Dropout(dropout)(layer)
    layer = Dense(1, name="out_layer")(layer)
    layer = Activation("sigmoid")(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model


def predict_result(X_test, Y_test, model, tok):
    test_sequences = tok.texts_to_sequences(X_test)
    test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)
    accr = model.evaluate(test_sequences_matrix, Y_test)
    print("Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}".format(accr[0], accr[1]))
    predict_labels = model.predict(test_sequences_matrix)

    # convert the range predicition into 0s or 1s
    Y_pred = [1 if i > 0.5 else 0 for i in predict_labels]

    print("Number Real news:", sum(Y_pred))
    print("Number Fake news:", len(Y_pred) - sum(Y_pred))
    print()
    print("Classification Report")
    print(classification_report(Y_test, Y_pred, target_names=["class 0", "class 1"]))
    print()
    print("Confusion Matrix")
    matrix = confusion_matrix(Y_test, Y_pred, labels=[0, 1])
    cm = pd.DataFrame(
        matrix,
        index=["class_0 pred", "class_1 pred"],
        columns=["class_0 True", "class_1 True"],
    )
    print(cm)
    return


earlystop = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy", verbose=1, patience=3
)


def trainer(X_train, Y_train, max_words, max_len, n_batchsize, n_epochs, model):
    tok = Tokenizer(
        num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True
    )
    tok.fit_on_texts(X_train)
    sequences = tok.texts_to_sequences(X_train)
    sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
    print("Found %s unique tokens." % len(sequences_matrix))
    model.summary()
    model.compile(loss="binary_crossentropy", optimizer=RMSprop(), metrics=["accuracy"])
    model.fit(
        sequences_matrix,
        Y_train,
        batch_size=n_batchsize,
        epochs=n_epochs,
        validation_split=0.2,
        callbacks=[earlystop],
    )

    return model, tok

### Read datasets

In [ ]:
# BS Detector
fake_news1 = read_news("/content/drive/MyDrive/dados_/fake.csv", "text")
# ISOT Dataset
fake_news2 = read_news("/content/drive/MyDrive/dados_/Fake_bisaillon.csv", "text")

# New York Times
real_news1 = read_news("/content/drive/MyDrive/dados_/df_2016.csv", "sentence")
# Reuters.com
real_news2 = read_news("/content/drive/MyDrive/dados_/True_bisaillon.csv", "text")

# shape of each dataset
print("Shape of each datasset")
print(
    "fake_news1: ",
    np.shape(fake_news1),
    "\nfake_news2: ",
    np.shape(fake_news2),
    "\nreal_news1: ",
    np.shape(real_news1),
    "\nreal_news2: ",
    np.shape(real_news2),
)
real_news1_smallset = real_news1.sample(n=15000, random_state=2, axis=0)
print("real_news1_smallset:", np.shape(real_news1_smallset))

# separate the location of the news from the news text (e.g. "WASHINGTON (Reuters)")
real_news2[["loc", "text"]] = real_news2["text"].str.split("-", 1, expand=True)

fake_news1:  (12999, 20) 
fake_news2:  (23481, 4) 
real_news1:  (105606, 3) 
real_news2:  (21417, 4)
real_news1 small set: (15000, 3)


### **Train model with 2 datasets**
> Datasets BS Detector and NY Times

In [ ]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(
    real_news1, fake_news1
)

#### Parameters tunning

In [ ]:
max_words = 1000
max_len = 300
n_batchsize = 128
n_epochs = 20
dropout = 0.2

#### Train

In [ ]:
model = RNN(max_tokens=max_words, max_len=max_len, dropout=dropout)


plot_model(model, "model.png")
trained_model, tok = trainer(
    X_train,
    Y_train,
    max_words=max_words,
    max_len=max_len,
    n_batchsize=n_batchsize,
    n_epochs=n_epochs,
    model=model,
)

Found 85691 unique tokens.
Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 500)]             0         
                                                                 
 embedding_10 (Embedding)    (None, 500, 50)           100000    
                                                                 
 lstm_10 (LSTM)              (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_20 (Activation)  (None, 256)               0         
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 out_layer (Dense)           (N

#### Predict - Valid set

In [ ]:
predict_result(X_valid, Y_valid, trained_model, tok)

112/112 [==============================] - 0s 4ms/step - loss: 0.1367 - accuracy: 0.9703
Test set
  Loss: 0.137
  Accuracy: 0.970
112/112 [==============================] - 1s 6ms/step
Number Real news: 1978
Number Fake news: 1592

Classification Report
              precision    recall  f1-score   support

     class 0       0.97      0.96      0.97      1606
     class 1       0.97      0.98      0.97      1964

    accuracy                           0.97      3570
   macro avg       0.97      0.97      0.97      3570
weighted avg       0.97      0.97      0.97      3570


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          1546            60
class_1 pred            46          1918


#### Test higher number of words and longer sentence lenght

In [ ]:
max_words = 5000
max_len = 1000
n_batchsize = 128
n_epochs = 20
dropout = 0.2

model_higher = RNN(max_tokens=max_words, max_len=max_len, dropout=dropout)

trained_model_higher, tok = trainer(
    X_train,
    Y_train,
    max_words=max_words,
    max_len=max_len,
    n_batchsize=n_batchsize,
    n_epochs=n_epochs,
    model=model_higher,
)
predict_result(X_valid, Y_valid, trained_model_higher, tok)

Found 118128 unique tokens.
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 1000)]            0         
                                                                 
 embedding_13 (Embedding)    (None, 1000, 50)          250000    
                                                                 
 lstm_13 (LSTM)              (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_26 (Activation)  (None, 256)               0         
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 out_layer (Dense)           (

The second parameter test has higher valeus for vocabulary size ( max_words) and lenght of sentences (max_len). This choice resulted in even higher accuracy, which could indicate overfit of the data, therefore the previous parameters were the one choosen.

#### Test lower number of words and setence lenght

In [ ]:
max_words = 300
max_len = 50
n_batchsize = 128
n_epochs = 20
dropout = 0.2

model_lower = RNN(max_tokens=max_words, max_len=max_len, dropout=dropout)

trained_model_lower, tok = trainer(
    X_train,
    Y_train,
    max_words=max_words,
    max_len=max_len,
    n_batchsize=n_batchsize,
    n_epochs=n_epochs,
    model=model_lower,
)
predict_result(X_valid, Y_valid, tok)

Found 20229 unique tokens.
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 50)]              0         
                                                                 
 embedding_8 (Embedding)     (None, 50, 50)            15000     
                                                                 
 lstm_8 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_16 (Activation)  (None, 256)               0         
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (No

### **Train model with 2 datasets with similar lenght**

15000 rows for real news and 12999 for fake news

Same datasets as before.


In [ ]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(
    real_news1_smallset, fake_news1
)

#### Parameters tunning

In [ ]:
max_words = 1000
max_len = 150
n_batchsize = 128
n_epochs = 20
dropout = 0.2

#### Train

In [ ]:
model = RNN(max_tokens=max_words, max_len=max_len, dropout=dropout)

trained_model, tok = trainer(
    X_train,
    Y_train,
    max_words=max_words,
    max_len=max_len,
    n_batchsize=n_batchsize,
    n_epochs=n_epochs,
    model=model,
)

Found 20229 unique tokens.
Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_11 (Embedding)    (None, 150, 50)           50000     
                                                                 
 lstm_11 (LSTM)              (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_22 (Activation)  (None, 256)               0         
                                                                 
 dropout_11 (Dropout)        (None, 256)               0         
                                                                 
 out_layer (Dense)           (N

#### Predict - Valid set

In [ ]:
predict_result(X_valid, Y_valid, trained_model, tok)

112/112 [==============================] - 1s 6ms/step - loss: 0.1759 - accuracy: 0.9770
Test set
  Loss: 0.176
  Accuracy: 0.977
112/112 [==============================] - 1s 5ms/step
Number Real news: 2018
Number Fake news: 1552

Classification Report
              precision    recall  f1-score   support

     class 0       0.99      0.96      0.97      1606
     class 1       0.97      0.99      0.98      1964

    accuracy                           0.98      3570
   macro avg       0.98      0.98      0.98      3570
weighted avg       0.98      0.98      0.98      3570


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          1538            68
class_1 pred            14          1950


### **Train model with  4 datasets**

> Datasets: 

Real: NEW York times and Reuters.com (ISOT dataset)

Fake: BS Detector and fake news from ISOT Dataset

In [ ]:
real_news = pd.DataFrame(
    real_news1["text"].append(real_news2["text"], ignore_index=True)
)
real_news = real_news.sample(frac=1, random_state=1, ignore_index=True)

fake_news = pd.DataFrame(
    fake_news1["text"].append(fake_news2["text"], ignore_index=True)
)
fake_news = fake_news.sample(frac=1, random_state=1, ignore_index=True)

X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(
    real_news, fake_news
)

#### Parameters tunning

In [ ]:
max_words = 1000
max_len = 150
n_batchsize = 128
n_epochs = 20
dropout = 0.2

#### Train model

In [ ]:
model = RNN(max_tokens=max_words, max_len=max_len, dropout=dropout)

trained_model, tok = trainer(
    X_train,
    Y_train,
    max_words=max_words,
    max_len=max_len,
    n_batchsize=n_batchsize,
    n_epochs=n_epochs,
    model=model,
)

Found 118128 unique tokens.
Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_12 (Embedding)    (None, 150, 50)           50000     
                                                                 
 lstm_12 (LSTM)              (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_24 (Activation)  (None, 256)               0         
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 out_layer (Dense)           (

#### Predict - Valid set

In [ ]:
predict_result(X_valid, Y_valid, trained_model, tok)

652/652 [==============================] - 3s 5ms/step - loss: 0.0590 - accuracy: 0.9808
Test set
  Loss: 0.059
  Accuracy: 0.981
652/652 [==============================] - 3s 4ms/step
Number Real news: 16135
Number Fake news: 4712

Classification Report
              precision    recall  f1-score   support

     class 0       0.96      0.96      0.96      4704
     class 1       0.99      0.99      0.99     16143

    accuracy                           0.98     20847
   macro avg       0.97      0.97      0.97     20847
weighted avg       0.98      0.98      0.98     20847


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          4508           196
class_1 pred           204         15939


#### Predict - test set

In [ ]:
predict_result(X_test, Y_test, trained_model, tok)

767/767 [==============================] - 4s 5ms/step - loss: 0.0547 - accuracy: 0.9825
Test set
  Loss: 0.055
  Accuracy: 0.982
767/767 [==============================] - 3s 4ms/step
Number Real news: 19151
Number Fake news: 5375

Classification Report
              precision    recall  f1-score   support

     class 0       0.96      0.96      0.96      5371
     class 1       0.99      0.99      0.99     19155

    accuracy                           0.98     24526
   macro avg       0.97      0.97      0.97     24526
weighted avg       0.98      0.98      0.98     24526


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          5158           213
class_1 pred           217         18938


In [11]:
# generate requirements.txt

from pip._internal.utils.misc import get_installed_distributions
import sys

def get_imported_packages():
    p = get_installed_distributions()
    p = {package.key:package.version for package in p}

    imported_modules = set(sys.modules.keys())
    
    imported_modules.remove('pip')

    modules = [(m, p[m]) for m in imported_modules if p.get(m, False)]

    return modules


def generate_requirements(filepath:str, modules):
    with open(filepath, 'w') as f:
        for module, version in modules:
            f.write(f"{module}=={version}\n")


generate_requirements('requirements.txt', get_imported_packages())